In [2]:
from flask import Flask, request, jsonify


In [3]:
import numpy as np


In [4]:
import torch


In [5]:
from PIL import Image


In [6]:
import torchvision.transforms as transforms

In [ ]:
# 加载模型和权重参数
model = torch.load('path_to_model.pth')
model.eval()

# 图像预处理函数
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image

# 创建Flask应用
app = Flask(__name__)

# 定义API接口
@app.route('/predict', methods=['POST'])
def predict():
    # 接收客户端传入的图片
    image_file = request.files['image']
    image = Image.open(image_file)

    # 图像预处理
    image = preprocess_image(image)

    # 执行数字识别
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        prediction = predicted.item()

    # 返回识别结果
    result = {'prediction': prediction}
    return jsonify(result)

# 启动应用
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)